In [1]:
"""
Huggingface datasets に JAQKETのCC-BY-SA-4.0 ライセンスのデータを登録する
"""

import json
import urllib.request
from dataclasses import dataclass

from datasets import load_dataset, Dataset, DatasetDict  # type: ignore

JAQKET_V1_URLS = {
    "aio_01_test": "https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_01_test.jsonl",
    "aio_01_dev": "https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_01_dev.jsonl",
    "aio_01_unused": "https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_01_unused.jsonl",
}


# jaqket v1
@dataclass
class JaqketQuestionV1:
    qid: str
    question: str
    answers: list[str]
    competition: str
    timestamp: str
    section: str
    original_question: str
    original_answer: str
    original_additional_info: str
    number: int


def load_jaqket_v1(urls):
    res: list[JaqketQuestionV1] = []
    for url in urls:
        with urllib.request.urlopen(url) as f:
            # f は 1行ごとに処理
            data = [json.loads(line.decode("utf-8")) for line in f]
        for d in data:
            # dict_keys(['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers'])
            res.append(JaqketQuestionV1(**d))
    return res


jaqket_data = {}

for name, url in JAQKET_V1_URLS.items():
    print(name, url)
    jaqket_data[name] = load_jaqket_v1([url])

aio_01_test https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_01_test.jsonl
aio_01_dev https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_01_dev.jsonl
aio_01_unused https://jaqket.s3.ap-northeast-1.amazonaws.com/data/aio_02/aio_01_unused.jsonl


In [2]:
jaqket_data["aio_01_test"][0]

JaqketQuestionV1(qid='QA20CAPR-1002', question='「課長」こと有野晋哉が様々なレトロゲームをクリアしていくという内容の、CS放送の人気番組は何でしょう?', answers=['ゲームセンターCX'], competition='第1回AI王', timestamp='2019/12/25', section='リーダーボードテスト問題', original_question='「課長」こと有野晋哉が様々なレトロゲームをクリアしていくという内容の、CS放送の人気番組は何でしょう？', original_answer='ゲームセンターCX', original_additional_info='', number='1002')

In [7]:
# pd
import pandas as pd

ds = DatasetDict()
for name, data in jaqket_data.items():
    print(name, len(data))
    df = pd.DataFrame(data)
    ds[name] = Dataset.from_pandas(df)
ds

aio_01_test 2000
aio_01_dev 1992
aio_01_unused 608


DatasetDict({
    aio_01_test: Dataset({
        features: ['qid', 'question', 'answers', 'competition', 'timestamp', 'section', 'original_question', 'original_answer', 'original_additional_info', 'number'],
        num_rows: 2000
    })
    aio_01_dev: Dataset({
        features: ['qid', 'question', 'answers', 'competition', 'timestamp', 'section', 'original_question', 'original_answer', 'original_additional_info', 'number'],
        num_rows: 1992
    })
    aio_01_unused: Dataset({
        features: ['qid', 'question', 'answers', 'competition', 'timestamp', 'section', 'original_question', 'original_answer', 'original_additional_info', 'number'],
        num_rows: 608
    })
})

In [8]:
# ds.push_to_hub("hotchpotch/jaqket_cc", data_dir="v2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/hotchpotch/jaqket_cc/commit/dec4cb9e72e14f60bb07073a3dd576be6b01bc69', commit_message='Upload dataset', commit_description='', oid='dec4cb9e72e14f60bb07073a3dd576be6b01bc69', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
ds["aio_01_dev"][100]

{'qid': 'QA20CAPR-0202',
 'question': '変わり種として、ネコが務めたり、タレントが1日だけ務めたりすることもある、鉄道駅における最高責任者のことを何というでしょう?',
 'answers': ['駅長'],
 'competition': '第1回AI王',
 'timestamp': '2019/12/25',
 'section': '開発データ問題 (dev1)',
 'original_question': '変わり種として、ネコが務めたり、タレントが１日だけ務めたりすることもある、鉄道駅における最高責任者のことを何というでしょう？',
 'original_answer': '駅長',
 'original_additional_info': '',
 'number': '202'}

In [12]:
from datasets import load_dataset


ds = load_dataset("hotchpotch/jaqket_cc")
ds["aio_01_dev"][0]

Generating aio_01_test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating aio_01_dev split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating aio_01_unused split:   0%|          | 0/608 [00:00<?, ? examples/s]

{'qid': 'QA20CAPR-0002',
 'question': '明治時代に西洋から伝わった「テーブル・ターニング」に起源を持つ占いの一種で、50音表などを記入した紙を置き、参加者全員の人差し指をコインに置いて行うのは何でしょう?',
 'answers': ['コックリさん'],
 'competition': '第1回AI王',
 'timestamp': '2019/12/25',
 'section': '開発データ問題 (dev1)',
 'original_question': '明治時代に西洋から伝わった「テーブル・ターニング」に起源を持つ占いの一種で、50音表などを記入した紙を置き、参加者全員の人差し指をコインに置いて行うのは何でしょう？',
 'original_answer': 'コックリさん',
 'original_additional_info': '',
 'number': '2'}